In [2]:
import base64
import datetime
import hashlib
import json
import random
import time

from Crypto.Cipher import DES3
import pymongo
import requests
from playwright.sync_api import sync_playwright
import logging


class Encrypt:
    @classmethod
    def date(cls):
        date = datetime.datetime.now()
        return date.strftime('%Y%m%d')

    @classmethod
    def des_decrypt(cls, str1, str2):
        return cls._des_decrypt(str1, str2, cls.date())

    @classmethod
    def des_encrypt(cls, str_, secret_key, iv):
        cipher_decrypt = DES3.new(secret_key.encode('utf8'), DES3.MODE_CBC, iv.encode('utf8'))
        r = cipher_decrypt.encrypt(str_.encode('utf8'))

        # des = triple_des(secret_key.encode(), CBC, iv.encode(), pad=None, padmode=PAD_PKCS5)
        return base64.b64encode(r)

    @classmethod
    def _des_decrypt(cls, str_, secret_key, iv):
        cipher_decrypt = DES3.new(secret_key.encode('utf8'), DES3.MODE_CBC, iv.encode('utf8'))
        # des = triple_des(secret_key.encode(), CBC, iv.encode(), pad=None, padmode=PAD_PKCS5)
        r = cipher_decrypt.decrypt(base64.b64decode(str_))

        return r.decode('utf8')[:-ord(r.decode('utf8')[-1])]


class Cipher:
    stamp = ''

    @classmethod
    def binary(cls):
        try:
            date = datetime.datetime.now()
            time_stamp = str(int(date.timestamp() * 1000 * 1000))
            cls.stamp = time_stamp
            date_str = date.strftime('%Y%m%d')
            random24 = cls.random(24)
            origin = random24 + date_str + Encrypt.des_encrypt(time_stamp, random24, date_str).decode()

            return cls.binary_string(origin)
        except Exception as e:
            print(e)
            return ''

    @classmethod
    def random(cls, i):
        str_ = ''
        c_arr = [chr(x) for x in (*range(48, 58), *range(97, 123), *range(65, 91))]
        for i2 in range(i):
            str_ += c_arr[int(random.random() * len(c_arr))]
        return str_

    @classmethod
    def binary_string(cls, text):
        result = ''
        for i, c in enumerate(text):
            if i != 0:
                result += ' '
            result += format(ord(c), 'b')
        return result


class WenshuCrawler:
    def __init__(self):
        self.detail = ''
        proxy_host = ""
        proxy_port = ""
        proxy_user = ""
        proxy_pass = ""
        proxy = "http://{0}:{1}".format(proxy_host, proxy_port)
        proxy_meta = "http://%(user)s:%(pass)s@%(host)s:%(port)s" % {
            "host": proxy_host,
            "port": proxy_port,
            "user": proxy_user,
            "pass": proxy_pass,
        }
        proxies = {
            "http": proxy_meta,
            "https": proxy_meta,
        }
        self.browser_proxy = {"server": proxy, "username": proxy_user, "password": proxy_pass}
        self.playwright = None
        self.browser = None
        self.context = None
        self.page = None

        self.start_browser()

        self.request = requests.session()
        # self.request.proxies = proxies
        self.request.headers = {"Accept": "application/json, text/javascript, */*; q=0.01",
                                "Accept-Encoding": "gzip, deflate, br",
                                "Accept-Language": "zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2",
                                "Cache-Control": "no-cache",
                                "Connection": "keep-alive",
                                "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
                                "Host": "wenshu.court.gov.cn",
                                "Origin": "https://wenshu.court.gov.cn",
                                "Pragma": "no-cache",
                                "Referer": "https://wenshu.court.gov.cn/website/wenshu/181217BMTKHNT2W0/index.html?pageId=597cf241392682ddc37653ce9add806e&s2=%E5%8C%97%E4%BA%AC&cprqStart=2020-05-06&cprqEnd=2020-05-06",
                                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 chromium/80.0",
                                "X-Requested-With": "XMLHttpRequest"}

        mongo_client = pymongo.MongoClient("")
        self.spider_db = mongo_client["spider"]
        self.user_name = ''
        self.password = ''

    def start_browser(self):
        self.playwright = sync_playwright().start()
        # self.browser = self.playwright.firefox.launch(headless=False, proxy=self.browser_proxy)
        self.browser = self.playwright.firefox.launch(headless=False)
        self.context = self.browser.new_context(user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 chromium/80.0')
        self.page = self.context.new_page()

    def close_browser(self):
        self.context.close()
        self.browser.close()
        self.playwright.stop()

    def restart_browser(self):
        self.close_browser()
        self.start_browser()

    def login(self):
        logging.info('login...')
        for i in range(10):
            try:
                self.page.goto("https://wenshu.court.gov.cn/website/wenshu/181010CARHS5BS3C/index.html?open=login")
                self.page.wait_for_selector('//iframe')
                time.sleep(2)
                self.page.frames[1].type('[placeholder="手机号码"]', self.user_name)
                time.sleep(2)
                self.page.frames[1].type('[placeholder="密码"]', self.password)
                self.page.frames[1].click('.button-primary')
                time.sleep(10)
                return True
            except:
                logging.info('登陆失败, 重新登陆')
                pass
        return False


    def get_search(self, condition):
        cookies = self.context.cookies()
        # print(cookies)
        cookies_dict = {cookie['name']: cookie['value'] for cookie in cookies}
        b = Cipher.binary()
        i = Cipher.stamp
        form_data_temp = {
            "id": i,
            "command": 'queryDoc',
            "params": {
                "pageNum": '1',
                "pageSize": '1000',
                "sortFields": "s50:desc",
                "ciphertext": b,  # 加密的随机24-bit秘钥+时间戳+triple DES加密后的二 进制码
                "devid": "d13a47f0ab0243bc9822f24751dcc049",
                "devtype": "1",
                "queryCondition": json.loads(condition)
            },
        }
        data = {
            'request': base64.b64encode(json.dumps(form_data_temp).encode()).decode()
        }
        url = 'http://wenshuapp.court.gov.cn/appinterface/rest.q4w'
        tunnel = random.randint(1, 10000)
        self.request.headers["Proxy-Tunnel"] = str(tunnel)
        response = self.request.post(url=url, timeout=30, data=data, cookies=cookies_dict)
        response_json = response.json()
        if 'content' not in response.json()['data'].keys():
            return 0
        result = response_json['data']['content']
        key = response_json['data']['secretKey']
        plain_text = Encrypt.des_decrypt(result, key)
        json_data = json.loads(plain_text)
        for index, item in enumerate(json_data['queryResult']['resultList']):
            self.spider_db['wenshu_id_id'].insert(item)
        return len(json_data['queryResult']['resultList'])

    def crawl_search(self):
        self.login()
        condition = json.dumps([{"key": "s21", "value": "产假"}, {"key": "s12", "value": "9461"}, {"key": "s8", "value": "03"}, {"key": "cprq", "value": "2021-01-01 TO 2021-06-01"}], ensure_ascii=False)
        self.crawl_search_condition(condition)
        condition = json.dumps([{"key": "s21", "value": "产假"}, {"key": "s12", "value": "9461"}, {"key": "s8", "value": "03"}, {"key": "cprq", "value": "2021-06-02 TO 2099-01-01"}], ensure_ascii=False)
        self.crawl_search_condition(condition)
        self.close_browser()

    def crawl_search_condition(self, condition):
        for i in range(10):
            # self.get_search(condition)
            try:
                data_count = self.get_search(condition)
                logging.info(f'running success : {condition} data_count : {data_count}')
                break
            except Exception as e:
                print(e)
                logging.info(f'running fail : {condition}')


    def get_wenshu_detail(self, wenshu_id):
        cookies = self.context.cookies()
        cookies_dict = {cookie['name']: cookie['value'] for cookie in cookies}
        data = {"docId": wenshu_id,
                "ciphertext": Cipher.binary(),
                "cfg": "com.lawyee.judge.dc.parse.dto.SearchDataDsoDTO@docInfoSearch", "__RequestVerificationToken": "9t0n3NYuIJPFU8VUPb6wjMKM"}
        url = 'https://wenshu.court.gov.cn/website/parse/rest.q4w'
        tunnel = random.randint(1, 10000)
        self.request.headers["Proxy-Tunnel"] = str(tunnel)
        response = self.request.post(url=url, timeout=30, data=data, cookies=cookies_dict)
        result = response.json()['result']
        key = response.json()['secretKey']
        plain_text = Encrypt.des_decrypt(result, key)
        json_data = json.loads(plain_text)
        print(json_data)
        return json_data

    def crawl_detail(self):
        self.login()
        try:
            self.page.goto('https://wenshu.court.gov.cn/website/wenshu/181109AWZA70BFK4/index.html')
        except:
            pass

        restart_error_count = 0
        for wenshu in self.spider_db['wenshu_id_id'].find():
            wenshu_id = wenshu['rowkey']
            for i in range(10):
                try:
                    json_data = self.get_wenshu_detail(wenshu_id)
                    self.upload_wenshu_detail(json_data, wenshu)
                    restart_error_count = 0
                    break
                except:
                    restart_error_count += 1
                    try:
                        self.page.goto('https://wenshu.court.gov.cn/website/wenshu/181109AWZA70BFK4/index.html')
                    except:
                        pass
            if restart_error_count > 20:
                logging.error('请求错误')
                break
        self.close_browser()

    def upload_wenshu_detail(self, json_data):
        data_item = dict()
        data_item['title'] = json_data.get('s1', '')
        data_item['case_number'] = json_data.get('s7', '')
        data_item['court_id'] = json_data.get('s3', '')
        data_item['court_name'] = json_data.get('s2', '')
        data_item['document_id'] = json_data.get('s5', '')
        data_item['judgement_date'] = json_data.get('s31', '')
        data_item['pub_date'] = json_data.get('s41', '')
        data_item['case_type'] = json_data.get('s8', '')
        data_item['judgement_type'] = json_data.get('s6', '')
        data_item['levelof_tria'] = json_data.get('s9', '')
        data_item['private_type'] = json_data.get('s32', '')
        data_item['cause'] = json_data.get('wenshuAy', '')
        data_item['parties'] = json_data.get('s17', '')
        data_item['keyword'] = json_data.get('s45', '')
        data_item['legal_basis'] = json_data.get('s47', '')
        data_item['rel_wenshu'] = json_data.get('rel_wenshu', '')
        data_item['view_count'] = json_data.get('view_count', '')
        data_item['directory'] = json_data.get('directory', '')
        data_item['globalnet'] = json_data.get('globalnet', '')
        data_item['header'] = json_data.get('s22', '')
        data_item['law_suit'] = json_data.get('s23', '')
        data_item['fact'] = json_data.get('s25', '')
        data_item['referee_view'] = json_data.get('s26', '')
        data_item['judgment_result'] = json_data.get('s27', '')
        data_item['footer'] = json_data.get('s28', '')
        data_item['appendix'] = json_data.get('s51', '')
        data_item['qw_content'] = json_data.get('qwContent', '')
        data_item['last_updatetime'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        data_item['doc_id'] = hashlib.md5((data_item['title'] + data_item['case_number'] + data_item['judgement_date']).encode()).hexdigest()
        data_item['spider'] = 'wenshu_spider'
        self.spider_db['wenshu_id_id_content'].insert(data_item)


if __name__ == '__main__':
    w = WenshuCrawler()
    w.crawl_search()
    w.crawl_detail()
    del w


ModuleNotFoundError: No module named 'Crypto'